# scraper test

In [2]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup

# from funcs import create_game_info, create_team_info, create_info_df, create_boxscores, merge_boxscores, change_dtypes, create_PIE, commit_data

# import pandas as pd
# import numpy as np
# import string
# import time
# import sqlite3
# import warnings
# warnings.filterwarnings('ignore')

# pd.set_option('display.max_columns', None)

%run funcs.py

In [3]:
# global variables

# used to create id strings later
base_url = 'https://www.basketball-reference.com'

season_gamecount = 1

precovid_seasons = ['1314', '1415', '1516', '1617', '1718', '1819']
precovid_url_years = ['2014', '2015', '2016', '2017', '2018', '2019']
postcovid_seasons = ['1920', '2021', '2122', '2223']
postcovid_url_years = ['2020', '2021', '2022', '2023']

# post_covid_season_dict = {'1920': {'month_len': 8, 'final_month_gamecount': 83},
#                           '2021': {'month_len': 6, 'final_month_gamecount': 140},
#                           '2122': {'month_len': 7, 'final_month_gamecount': 83},
#                           '2223': {'month_len': 7, 'final_month_gamecount': 72}
#                          }

# used to create sql database table columns

info_columns = ['game_id', 'season', 'date', 'type', 'away_team', 'away_score', 'home_team', 'home_score', 'result']
num_columns = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
                'FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

basic_stats = ['MP', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
# advanced_stats = ['FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

rand = 4

In [4]:
# connect to sql database
conn = sqlite3.connect('data/temp/quarterly_test-temp.sqlite')
driver = webdriver.Chrome(service=svc,options=options)
time.sleep(rand)

In [5]:
game_type = 'regular'

In [6]:
month_url = 'https://www.basketball-reference.com/leagues/NBA_2017_games-april.html'
season_id = '1617'


driver.get(month_url)
time.sleep(rand)
month_src = driver.page_source
month_parser = BeautifulSoup(month_src, 'lxml')
month_table = month_parser.find('div', attrs = {'class': 'table_container is_setup'})

# check if final month (apr). if true, set limit for game_urls before playoffs start
final_reg_game_row = None
splits = month_table.findAll('tr', attrs = {'class': 'thead'})
for split in splits:
    if 'Playoffs' in split.text:
        final_reg_game_row = int(split['data-row'])
        break

In [7]:
# get partial urls of every game in the month (if apr, stop before playoffs)
if final_reg_game_row == None:
    
    game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
    game_urls = [base_url + url.a['href'] for url in game_partial_urls]

    for i in range(len(game_urls)):

        driver.quit()
        time.sleep(rand)
        driver = webdriver.Chrome(service=svc,options=options)
        time.sleep(rand)
        driver.maximize_window()
        driver.get(reg_game_urls[i])
        time.sleep(rand)
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')

        # game_info database:

        id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
        game_info = create_game_info(url=reg_game_urls[i],
                                     season_id=season_id,
                                     season_gamecount=season_gamecount,
                                     game_type=game_type)
        # will use game_id with create_boxscores()
        game_id = game_info[0]
        team_info = create_team_info(id_table)
        # will use team_ids with merge_boxscores()
        team_ids = [team_info[0], team_info[2]]

        info_df = create_info_df(game_info=game_info,
                                 team_info=team_info,
                                 info_columns=info_columns)
        # write game info to sql database
        info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

        # team/player databases:

        # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
        stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

        player_box_list = [None, None, None, None]
        team_box_list = [None, None, None, None]

        # create team and player boxscores
        for i in range(len(stat_tables)):
            # split player and team boxscores
            player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

        # team_stats database:

        # combine boxscore and advanced boxscore for each team
        away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
        team_boxes = pd.concat([away_team_box, home_team_box])
        team_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        team_boxes = change_dtypes(team_boxes, num_columns)
        # write to sql database
        team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

        # player_stats database:

        # combine boxscore and advanced boxscore for each team
        away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
        player_boxes = pd.concat([away_player_box, home_player_box])
        player_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        player_boxes = change_dtypes(player_boxes, num_columns)
        # create team totals for PIE calculation
        totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
        # add PIE column to player boxscore
        player_boxes = create_PIE(player_boxes, totals)
        # write to sql database
        player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
        
        
        
        button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

        buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
        del buttons[6]
        del buttons[3]
        del buttons[0]
        
        game_length = len(buttons)
        
        button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
        game_button_codes = button_codes[:game_length]
        
        for team_id in team_ids:
            for period in game_button_codes:
                
                id_string = f'box-{team_id}-{period}-basic'
                period_table = parser.find('table', attrs = {'id': id_string})
                
                player_box, team_box = create_boxscores(period_table, game_id=game_id)
            
                # team_stats database:

                team_box.rename(columns={'Starters': 'team'}, inplace=True)
                team_box['team'] = team_id
                team_box.insert(loc=1, column='period', value=period)
                
                team_box.reset_index(drop=True, inplace=True)
                team_box = change_dtypes(team_box, basic_stats)

                team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                # player_stats database:

                player_box.insert(loc=2, column='team', value=team_id)
                player_box.insert(loc=1, column='period', value=period)

                player_box.reset_index(drop=True, inplace=True)
                player_box = change_dtypes(player_box, basic_stats)

                player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

        # increase gamecount to create next game_id
        season_gamecount += 1

elif final_reg_game_row != None:
    game_partial_urls = month_table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
    reg_game_urls = [base_url + url.a['href'] for url in game_partial_urls[:final_reg_game_row]]
    playoff_game_urls = [base_url + url.a['href'] for url in game_partial_urls[final_reg_game_row:]]
    
    for i in range(len(reg_game_urls)):

        driver.quit()
        time.sleep(rand)
        driver = webdriver.Chrome(service=svc,options=options)
        time.sleep(rand)
        driver.maximize_window()
        driver.get(reg_game_urls[i])
        time.sleep(rand)
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')

        # game_info database:

        id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
        game_info = create_game_info(url=reg_game_urls[i],
                                     season_id=season_id,
                                     season_gamecount=season_gamecount,
                                     game_type=game_type)
        # will use game_id with create_boxscores()
        game_id = game_info[0]
        team_info = create_team_info(id_table)
        # will use team_ids with merge_boxscores()
        team_ids = [team_info[0], team_info[2]]

        info_df = create_info_df(game_info=game_info,
                                 team_info=team_info,
                                 info_columns=info_columns)
        # write game info to sql database
        info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

        # team/player databases:

        # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
        stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

        player_box_list = [None, None, None, None]
        team_box_list = [None, None, None, None]

        # create team and player boxscores
        for i in range(len(stat_tables)):
            # split player and team boxscores
            player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

        # team_stats database:

        # combine boxscore and advanced boxscore for each team
        away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
        team_boxes = pd.concat([away_team_box, home_team_box])
        team_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        team_boxes = change_dtypes(team_boxes, num_columns)
        # write to sql database
        team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

        # player_stats database:

        # combine boxscore and advanced boxscore for each team
        away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
        player_boxes = pd.concat([away_player_box, home_player_box])
        player_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        player_boxes = change_dtypes(player_boxes, num_columns)
        # create team totals for PIE calculation
        totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
        # add PIE column to player boxscore
        player_boxes = create_PIE(player_boxes, totals)
        # write to sql database
        player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
        
        
        
        button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

        buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
        del buttons[6]
        del buttons[3]
        del buttons[0]
        
        game_length = len(buttons)
        
        button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
        game_button_codes = button_codes[:game_length]
        
        for team_id in team_ids:
            for period in game_button_codes:
                
                id_string = f'box-{team_id}-{period}-basic'
                period_table = parser.find('table', attrs = {'id': id_string})
                
                player_box, team_box = create_boxscores(period_table, game_id=game_id)
            
                # team_stats database:

                team_box.rename(columns={'Starters': 'team'}, inplace=True)
                team_box['team'] = team_id
                team_box.insert(loc=1, column='period', value=period)
                
                team_box.reset_index(drop=True, inplace=True)
                team_box = change_dtypes(team_box, basic_stats)

                team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                # player_stats database:

                player_box.insert(loc=2, column='team', value=team_id)
                player_box.insert(loc=1, column='period', value=period)

                player_box.reset_index(drop=True, inplace=True)
                player_box = change_dtypes(player_box, basic_stats)

                player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)

        # increase gamecount to create next game_id
        season_gamecount += 1
        
    game_type = 'playoff'
    
    for i in range(len(playoff_game_urls)):

        driver.quit()
        time.sleep(rand)
        driver = webdriver.Chrome(service=svc,options=options)
        time.sleep(rand)
        driver.get(reg_game_urls[i])
        time.sleep(rand)
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')

        # game_info database:

        id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
        game_info = create_game_info(url=reg_game_urls[i],
                                     season_id=season_id,
                                     season_gamecount=season_gamecount,
                                     game_type=game_type)
        # will use game_id with create_boxscores()
        game_id = game_info[0]
        team_info = create_team_info(id_table)
        # will use team_ids with merge_boxscores()
        team_ids = [team_info[0], team_info[2]]

        info_df = create_info_df(game_info=game_info,
                                 team_info=team_info,
                                 info_columns=info_columns)
        # write game info to sql database
        info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

        # team/player databases:

        # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
        stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

        player_box_list = [None, None, None, None]
        team_box_list = [None, None, None, None]

        # create team and player boxscores
        for i in range(len(stat_tables)):
            # split player and team boxscores
            player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

        # team_stats database:

        # combine boxscore and advanced boxscore for each team
        away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
        team_boxes = pd.concat([away_team_box, home_team_box])
        team_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        team_boxes = change_dtypes(team_boxes, num_columns)
        # write to sql database
        team_boxes.to_sql('team_stats_game', con=conn, if_exists='append', index=False)

        # player_stats database:

        # combine boxscore and advanced boxscore for each team
        away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
        player_boxes = pd.concat([away_player_box, home_player_box])
        player_boxes.reset_index(drop=True, inplace=True)
        # prepare numeric data
        player_boxes = change_dtypes(player_boxes, num_columns)
        # create team totals for PIE calculation
        totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
        # add PIE column to player boxscore
        player_boxes = create_PIE(player_boxes, totals)
        # write to sql database
        player_boxes.to_sql('player_stats_game', con=conn, if_exists='append', index=False)
        
        # quarterly boxscores:
        button_banner = driver.find_element(By.CLASS_NAME, 'filter.switcher')

        # button order: Game, Q1, Q2, H1, Q3, Q4, H2, (OT1, OT2, ...)
        buttons = button_banner.find_elements(By.CLASS_NAME, 'sr_preset')
        
        del buttons[6]
        del buttons[3]
        del buttons[0]
        
        game_length = len(buttons)
        
        button_codes = ['q1', 'q2', 'q3', 'q4', 'ot1', 'ot2', 'ot3', 'ot4', 'ot5', 'ot6']
        game_button_codes = button_codes[:game_length]
        
        for team_id in team_ids:
            for period in game_button_codes:
                
                id_string = f'box-{team_id}-{period}-basic'
                period_table = parser.find('table', attrs = {'id': id_string})
                
                player_box, team_box = create_boxscores(period_table, game_id=game_id)
            
                # team_stats database:

                team_box.rename(columns={'Starters': 'team'}, inplace=True)
                team_box['team'] = team_id
                team_box.insert(loc=1, column='period', value=period)

                team_box = change_dtypes(team_box, basic_stats)

                team_box.to_sql('team_stats_quarterly', con=conn, if_exists='append', index=False)

                # player_stats database:

                player_box.insert(loc=2, column='team', value=team_id)
                player_box.insert(loc=1, column='period', value=period)

                player_box = change_dtypes(player_box, basic_stats)

                player_box.to_sql('player_stats_quarterly', con=conn, if_exists='append', index=False)
        
        # increase gamecount to create next game_id
        season_gamecount += 1

ValueError: All objects passed were None

In [ ]:
id_string = f'box-LAL-q1-basic'
period_table = parser.find('table', attrs = {'id': id_string})

In [38]:
query = 'SELECT * from player_stats_quarterly'
game_info = pd.read_sql(query, con=conn)
# account for team name change following the 2013-2014 season
# game_info['away_team'].replace('CHA', 'CHO', inplace=True)
# game_info['home_team'].replace('CHA', 'CHO', inplace=True)
game_info

,game_id,period,player,team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
0,161704010001,q1,Jordan Clarkson,LAL,12:00,3.0,7.0,.429,1.0,3.0,.333,0.0,0.0,-99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,-6.0
1,161704010001,q1,D'Angelo Russell,LAL,9:19,0.0,3.0,.000,0.0,3.0,.000,0.0,0.0,-99,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,-3.0
2,161704010001,q1,Brandon Ingram,LAL,7:49,2.0,5.0,.400,0.0,0.0,-99,0.0,2.0,.000,0.0,2.0,2.0,1.0,0.0,0.0,1.0,1.0,4.0,-1.0
3,161704010001,q1,Larry Nance Jr.,LAL,7:23,1.0,2.0,.500,0.0,0.0,-99,0.0,0.0,-99,1.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,2.0,-3.0
4,161704010001,q1,Julius Randle,LAL,3:40,0.0,2.0,.000,0.0,0.0,-99,0.0,0.0,-99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,161704120091,q4,Patricio Garino,ORL,5:34,0.0,0.0,-99,0.0,0.0,-99,0.0,0.0,-99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0
9521,161704120091,q4,Marcus Georges-Hunt,ORL,9:34,0.0,0.0,-99,0.0,0.0,-99,4.0,4.0,1.000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,-4.0
9522,161704120091,q4,Stephen Zimmerman,ORL,8:52,1.0,2.0,.500,0.0,0.0,-99,0.0,0.0,-99,0.0,1.0,1.0,0.0,0.0,1.0,1.0,3.0,2.0,-6.0
9523,161704120091,q4,Bismack Biyombo,ORL,-99,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 23 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   game_id  746 non-null    int64  
 1   period   746 non-null    object 
 2   team     746 non-null    object 
 3   MP       746 non-null    object 
 4   FG       746 non-null    float64
 5   FGA      746 non-null    float64
 6   FG%      746 non-null    object 
 7   3P       746 non-null    float64
 8   3PA      746 non-null    float64
 9   3P%      746 non-null    object 
 10  FT       746 non-null    float64
 11  FTA      746 non-null    float64
 12  FT%      746 non-null    object 
 13  ORB      746 non-null    float64
 14  DRB      746 non-null    float64
 15  TRB      746 non-null    float64
 16  AST      746 non-null    float64
 17  STL      746 non-null    float64
 18  BLK      746 non-null    float64
 19  TOV      746 non-null    float64
 20  PF       746 non-null    float64
 21  PTS      746 non

In [36]:
game_info.loc[:,'FG':] = game_info.loc[:,'FG':].astype('float64')

In [37]:
game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 23 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   game_id  746 non-null    int64  
 1   period   746 non-null    object 
 2   team     746 non-null    object 
 3   MP       746 non-null    object 
 4   FG       746 non-null    float64
 5   FGA      746 non-null    float64
 6   FG%      746 non-null    float64
 7   3P       746 non-null    float64
 8   3PA      746 non-null    float64
 9   3P%      746 non-null    float64
 10  FT       746 non-null    float64
 11  FTA      746 non-null    float64
 12  FT%      746 non-null    float64
 13  ORB      746 non-null    float64
 14  DRB      746 non-null    float64
 15  TRB      746 non-null    float64
 16  AST      746 non-null    float64
 17  STL      746 non-null    float64
 18  BLK      746 non-null    float64
 19  TOV      746 non-null    float64
 20  PF       746 non-null    float64
 21  PTS      746 non

In [ ]:
for i in range(len(postcovid_seasons)):
    
    season_id = postcovid_seasons[i]
    season_gamecount = 1
    start_url = 'https://www.basketball-reference.com/leagues/NBA_' + postcovid_url_years[i] + '_games.html'
    
    # open the season schedule page
    driver.get(start_url)
    delay
    src = driver.page_source
    # create beautiful soup object from html/xml
    parser = BeautifulSoup(src, 'lxml')
    
    # every month from the season
    months = parser.find('div', attrs = {'class': 'filter'})
    # partial urls for each month
    links = months.findAll('a')
    # full urls for each month
    month_links = [base_url + link['href'] for link in links]
    # only include regular season months
    month_links = month_links[0:post_covid_season_dict[season_id]['month_len']]
    
    for month_url in month_links:
        
        driver.quit()
        driver = webdriver.Chrome(ChromeDriverManager().install())
        delay
        
        driver.get(month_url)
        delay
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')
        table = parser.find('div', attrs = {'class': 'table_container is_setup'})
        
      
        # check if final month. if true, set limit for game_urls before playoffs start        
        if month_url != month_links[-1]:
            game_partial_urls = table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
        else:
            play_in = table.find('td', string='Play-In Game').find_parent()
            play_in_row = int(play_in['data-row'])
            body= table.find('tbody')
            all_rows = body.findAll('tr', limit=play_in_row)
            
            game_rows = []
            for row in all_rows:
                try:
                    row['class']
                except KeyError:
                    game_rows.append(row)
                else:
                    pass
            game_partial_urls = [row.find(attrs = {'class': 'center', 'data-stat': 'box_score_text'}) for row in game_rows]
        
        # get game_urls
        game_urls = [base_url + url.a['href'] for url in game_partial_urls]
        
        # open every game url, retrieve and manipulate data, add to sql database
        for i in range(len(game_urls)):
    
            driver.get(game_urls[i])
            delay
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')
            
            # game_info database:
            
            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount)
            # will use game_id with create_boxscores()
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            # will use team_ids with merge_boxscores()
            team_ids = [team_info[0], team_info[2]]
            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns)
            # write to sql database
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            # team/player databases:
            
            # 4 boxscore tables : away_box, away_box_adv, home_box, home_box_adv
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
            
            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            # create team and player boxscores
            for i in range(len(stat_tables)):
                # split player and team boxscores
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)
            
            # team_stats database:
            
            # combine boxscore and advanced boxscore for each team
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            team_boxes = change_dtypes(team_boxes, num_columns)
            # write to sql database
            team_boxes.to_sql('team_stats', con=conn, if_exists='append', index=False)
            
            # player_stats database:
            
            # combine boxscore and advanced boxscore for each team
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            # prepare numeric data
            player_boxes = change_dtypes(player_boxes, num_columns)
            # create team totals for PIE calculation
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())
            # add PIE column to player boxscore
            player_boxes = create_PIE(player_boxes, totals)
            # write to sql database
            player_boxes.to_sql('player_stats', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1